In [ ]:
using Revise
using DrWatson
using CSV
using DataFrames
using TableView
using StatsBase

using Plots; pyplot()

import PMDPs, PMDPs.CountingProcesses
using Random
using StaticArrays
using RandomNumbers
using RandomNumbers.Xorshifts
using POMDPSimulators
using MCTS

ENV["COLUMNS"]=200

In [3]:
tam = datadir("bus_data", "Praha_Londyn_result.csv")
zpet = datadir("bus_data", "Londyn_Praha_result.csv")

"/home/mrkos/projects/PMDPs/data/bus_data/Londyn_Praha_result.csv"

In [20]:
df = DataFrame(CSV.File(tam));

showtable(df)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[""], Dict{Symbol,Any}(:style => Dict("height" => "0","padding" => "0","background-color" => "#F5F7F7","color" => "#1c1f20","font-family" => "-apple-system, BlinkMacSystemFont, \"Segoe UI\", Roboto, Oxygen-Sans, Ubuntu, Cantarell, \"Helvetica Neue\", sans-serif"))), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "grid-f65b6e6c",:className => "ag-theme-balham",:style => Dict("height" => "500px","width" => "100%")))], Dict{Symbol,Any}()), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("onCellKeyDown" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onRowClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellDoubleClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellFocused" => (Observable{Any} with 0 listeners. Value:
nothing, nothing)), Set{String}(), nothing, WebIO.Asset[WebIO.Asset("js", nothing, "/home/mrkos/.julia/packages/TableView/CT3WZ/src/rowNumberRenderer.js"), WebIO.Asset("js", nothing, "/home/mrkos/.julia/packages/TableView/CT3WZ/deps/ag-grid-25.0.0/ag-grid.js"), WebIO.Asset("css", nothing, "/home/mrkos/.julia/packages/TableView/CT3WZ/deps/ag-grid-25.0.0/ag-grid.css"), WebIO.Asset("css", nothing, "/home/mrkos/.julia/packages/TableView/CT3WZ/deps/ag-grid-25.0.0/ag-grid-light.css"), WebIO.Asset("css", nothing, "/home/mrkos/.julia/packages/TableView/CT3WZ/deps/ag-grid-25.0.0/ag-grid-dark.css")], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{WebIO.AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007ff10965a980, Task (runnable) @0x00007ff10965a980), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (RowNumberRenderer,agGrid){var gridOptions={\"onCellFocused\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellFocused\",\"scope\":\"9674478247246771097\",\"id\":\"9357415231482697197\",\"type\":\"observable\"},x)}),\"onCellValueChanged\":(function (){return }),\"onCellDoubleClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellDoubleClicked\",\"scope\":\"9674478247246771097\",\"id\":\"3184185749687998503\",\"type\":\"observable\"},x)}),\"multiSortKey\":\"ctrl\",\"columnDefs\":[{\"filter\":false,\"editable\":false,\"headerTooltip\":\"\",\"field\":\"__row__\",\"headerName\":\"Row\",\"sortable\":true,\"type\":\"numericColumn\",\"cellRenderer\":\"rowNumberRenderer\",\"resizable\":true},{\"filter\":\"agNumberColumnFilter\",\"editable\":false,\"headerTooltip\":\"Int64\",\"field\":\"n_seats\",\"headerName\":\"n_seats\",\"sortable\":true,\"type\":\"numericColumn\",\"resizable\":true},{\"filter\":\"agNumberColumnFilter\",\"editable\":false,\"headerTooltip\":\"Float64\",\"field\":\"mean_h_to_depart\",\"headerName\":\"mean_h_to_depart\",\"sortable\":true,\"type\":\"numericColumn\",\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"o_d\",\"headerName\":\"o_d\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"bus_id\",\"headerName\":\"bus_id\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":\"agNumberColumnFilter\",\"editable\":false,\"headerTooltip\":\"Int64\",\"field\":\"occupancy_Praha-Lucemburk\",\"headerName\":\"occupancy_Praha-Lucemburk\",\"sortable\":true,\"type\":\"numericColumn\",\"resizable\":true},{\"filter\":

In [21]:
A, B, C, D = "Praha", "Lucemburk", "Brussels", "Londyn"
product_map = Dict(
    "$A-$B" => [1,0,0],
    "$B-$C" => [0,1,0],
    "$C-$D" => [0,0,1], 
    "$A-$C" => [1,1,0],
    "$B-$D" => [0,1,1],
    "$A-$D" => [1,1,1],
)

df.prod = [product_map[od] for od in df.o_d]

491-element Array{Array{Int64,1},1}:
 [1, 1, 1]
 [1, 1, 1]
 [1, 1, 0]
 [1, 1, 0]
 [0, 0, 1]
 [0, 1, 0]
 [1, 1, 0]
 [1, 0, 0]
 [1, 1, 0]
 [1, 0, 0]
 [1, 1, 1]
 [0, 0, 1]
 [1, 0, 0]
 ⋮
 [1, 1, 0]
 [1, 1, 1]
 [1, 0, 0]
 [1, 1, 0]
 [1, 1, 1]
 [1, 1, 0]
 [1, 1, 0]
 [1, 1, 0]
 [1, 1, 0]
 [1, 1, 1]
 [1, 1, 0]
 [1, 1, 1]

In [22]:
maximum(df[!, 5]), maximum(df[!, 6]), maximum(df[!, 7])

(55, 55, 52)

In [23]:
function stretch_hist(h, n_timesteps)
    timesteps_per_bin = Int64(ceil(n_timesteps/length(h.weights)[1]))
    vcat([ones(timesteps_per_bin)*w/timesteps_per_bin for w in h.weights]...)[1:n_timesteps]
end

stretch_hist (generic function with 1 method)

In [75]:
gps = groupby(df, :o_d)

# histogram(gr.mean_h_to_depart)
time_range = 0:100:6000
timestep_range = 1:1000
expected_requests = 80

# Products
spe = timestep_range[end]
products = [PMDPs.Product{length(first(g.prod))}(first(g.prod), spe) for g in gps]

# Demand
hs = [fit(Histogram, gr.mean_h_to_depart, time_range) for gr in gps]
req_counts = hcat([reverse(stretch_hist(h, length(timestep_range))) for h in hs]...)'

probs =  req_counts./sum(req_counts)*expected_requests

6×1000 Array{Float64,2}:
 0.0         0.0         0.0         0.0         0.0         0.0         0.0         0.0         …  0.239779    0.239779    0.239779    0.239779    0.239779    0.239779    0.239779    0.239779
 0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118     0.220597    0.220597    0.220597    0.220597    0.220597    0.220597    0.220597    0.220597
 0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118     0.0575471   0.0575471   0.0575471   0.0575471   0.0575471   0.0575471   0.0575471   0.0575471
 0.0         0.0         0.0         0.0         0.0         0.0         0.0         0.0            0.0863206   0.0863206   0.0863206   0.0863206   0.0863206   0.0863206   0.0863206   0.0863206
 0.0         0.0         0.0         0.0         0.0         0.0         0.0         0.0            0.0383647   0.0383647   0.0383647   0.0383647   0.0383647   0.0383647   0.0383647   0.0383647
 0.0   

In [76]:
"""Time 2 timestep"""
function t2ts(time, time_range, timestep_range)
    ts = timestep_range[1] + (time - time_range[1]) * (timestep_range[end]-timestep_range[1])/(time_range[end]-time_range[1])
    Int64(round(ts))
end

df.ts = [t2ts(time_range[end]-h, time_range, timestep_range) for h in df.mean_h_to_depart];

In [77]:
df[shuffle(1:8), :]

,n_seats,mean_h_to_depart,o_d,bus_id,occupancy_Praha-Lucemburk,occupancy_Lucemburk-Brussels,occupancy_Brussels-Londyn,prod,ts,iₚ
,Int64,Float64,String,String,Int64,Int64,Int64,Array…,Int64,Int64
1,1,1710.0,Praha-Londyn,00a68a3eeffd9be277c9a3ec27e3bb7b,8,8,7,"[1, 1, 1]",715,1
2,1,293.0,Praha-Brussels,00f921d978ff427b844b2364a8bff72c,9,9,8,"[1, 1, 0]",951,2
3,4,1878.0,Praha-Londyn,00a68a3eeffd9be277c9a3ec27e3bb7b,6,6,6,"[1, 1, 1]",687,1
4,1,269.0,Brussels-Londyn,00f921d978ff427b844b2364a8bff72c,9,9,9,"[0, 0, 1]",955,3
5,1,1710.0,Praha-Brussels,00a68a3eeffd9be277c9a3ec27e3bb7b,8,8,7,"[1, 1, 0]",715,2
6,1,181.0,Praha-Lucemburk,00f921d978ff427b844b2364a8bff72c,11,11,9,"[1, 0, 0]",970,5
7,1,257.0,Lucemburk-Brussels,00f921d978ff427b844b2364a8bff72c,9,10,9,"[0, 1, 0]",957,4
8,1,197.0,Praha-Brussels,00f921d978ff427b844b2364a8bff72c,10,11,9,"[1, 1, 0]",967,2


# Preparing PP

In [165]:
nᵣ = 3
c = 55
T = 2000


time_range = 0:100:6000
timestep_range = 1:T
expected_requests = 160

res_budget_μ = 5.

# objective = :revenue
objective = :utilization

:utilization

In [148]:
gps = groupby(df, :o_d)

# Products
spe = timestep_range[end]
products = [PMDPs.Product{length(first(g.prod))}(first(g.prod), spe) for g in gps]

# Demand
hs = [fit(Histogram, gr.mean_h_to_depart, time_range) for gr in gps]
req_counts = hcat([reverse(stretch_hist(h, length(timestep_range))) for h in hs]...)'

probs =  req_counts./sum(req_counts)*expected_requests

probs

6×2000 Array{Float64,2}:
 0.0         0.0         0.0         0.0         0.0         0.0         0.0         0.0         …  0.239779    0.239779    0.239779    0.239779    0.239779    0.239779    0.239779    0.239779
 0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118     0.220597    0.220597    0.220597    0.220597    0.220597    0.220597    0.220597    0.220597
 0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118  0.00959118     0.0575471   0.0575471   0.0575471   0.0575471   0.0575471   0.0575471   0.0575471   0.0575471
 0.0         0.0         0.0         0.0         0.0         0.0         0.0         0.0            0.0863206   0.0863206   0.0863206   0.0863206   0.0863206   0.0863206   0.0863206   0.0863206
 0.0         0.0         0.0         0.0         0.0         0.0         0.0         0.0            0.0383647   0.0383647   0.0383647   0.0383647   0.0383647   0.0383647   0.0383647   0.0383647
 0.0   

In [149]:
hcat([h.weights for h in hs]...)

60×6 Array{Int64,2}:
 25  23  6  9   4  1
 24  35  8  9  12  1
 13  19  9  3   5  2
  8  11  4  0   4  1
  3   7  1  1   2  0
  4   3  1  0   1  2
  2   4  0  0   0  0
  5   5  2  2   0  0
  3   7  2  2   2  1
  6   7  1  1   1  0
  4   2  1  1   0  0
  2   4  0  0   0  0
  1   1  1  1   2  0
  ⋮                ⋮
  0   0  0  0   0  0
  1   3  0  0   0  0
  7   5  1  2   0  0
  1   1  0  0   0  0
  0   0  0  0   0  0
  0   0  0  0   0  0
  0   0  0  0   0  0
  1   2  0  0   1  0
  4   3  0  1   0  0
  4   5  0  0   1  0
  0   1  1  0   0  0
  0   0  0  0   0  0

In [150]:
D = PMDPs.CountingProcesses.NonhomogenousBernoulliScheme(probs)
for i in 1:size(probs)[2]
    @assert sum(D[i].p)==1.
end

In [166]:
"""
Initial capacity is the same for every resource
"""
c₀ = [c for v in 1:nᵣ]

"""
Resource selling period ends should end one by one at each timestep.
"""
resource_selling_period_ends = [T for i in 1:nᵣ]
# for i::Int64 in 1:ceil(nᵣ/2)
#     resource_selling_period_ends[1:(end-i)].-=1
# end

"""
Products
"""
P = products


"""
Demand
"""
rnd = Xorshift1024Plus()
D = PMDPs.CountingProcesses.NonhomogenousBernoulliScheme(probs)

"""
For the budgets, we assume normal distribution per resource.
"""
B = PMDPs.normal_budgets_per_resource(P, res_budget_μ, res_budget_μ/2)

"""
Actions
"""
A = PMDPs.action_space(P, res_budget_μ)

pp = PMDPs.PMDPProblem(P, c₀, D, B, A, objective)

PMDPs.PMDPProblem{6,3,10}(PMDPs.Product[[1, 1, 1], [1, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1]], [55, 55, 55], 2000, PMDPs.CountingProcesses.NonhomogenousBernoulliScheme([0.0 0.0 … 0.23977940294928285 0.23977940294928285; 0.009591176117971313 0.009591176117971313 … 0.2205970507133402 0.2205970507133402; … ; 0.0 0.0 … 0.03836470447188525 0.03836470447188525; 0.0 0.0 … 0.009591176117971313 0.009591176117971313]), Distributions.Distribution[Distributions.Normal{Float64}(μ=15.0, σ=2.5), Distributions.Normal{Float64}(μ=10.0, σ=2.5), Distributions.Normal{Float64}(μ=5.0, σ=2.5), Distributions.Normal{Float64}(μ=5.0, σ=2.5), Distributions.Normal{Float64}(μ=5.0, σ=2.5), Distributions.Normal{Float64}(μ=10.0, σ=2.5)], [0.0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0, 1.7976931348623157e308], :utilization)

## Prepare traces

In [168]:
mg = PMDPs.PMDPg(pp)

PMDPs.PMDPg(PMDPs.PMDPProblem{6,3,10}(PMDPs.Product[[1, 1, 1], [1, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1]], [55, 55, 55], 2000, PMDPs.CountingProcesses.NonhomogenousBernoulliScheme([0.0 0.0 … 0.23977940294928285 0.23977940294928285; 0.009591176117971313 0.009591176117971313 … 0.2205970507133402 0.2205970507133402; … ; 0.0 0.0 … 0.03836470447188525 0.03836470447188525; 0.0 0.0 … 0.009591176117971313 0.009591176117971313]), Distributions.Distribution[Distributions.Normal{Float64}(μ=15.0, σ=2.5), Distributions.Normal{Float64}(μ=10.0, σ=2.5), Distributions.Normal{Float64}(μ=5.0, σ=2.5), Distributions.Normal{Float64}(μ=5.0, σ=2.5), Distributions.Normal{Float64}(μ=5.0, σ=2.5), Distributions.Normal{Float64}(μ=10.0, σ=2.5)], [0.0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0, 1.7976931348623157e308], :utilization), Bool[0, 0, 0], 7)

In [169]:
# add product id column
pid = Dict(zip(P, 1:6))
df.iₚ = [pid[pr] for pr in df.prod];

In [170]:
function shift_duplicate_ts!(requests)
    sort!(requests, :ts)
    ts = requests.ts
    for i in length(ts):-1:1
        while i> 2 && ts[i] <= ts[i-1]
            ts[i-1]=ts[i-1]-1
        end
    end
end


function generate_bus_trace(df, pp, rnd)
    requests = copy(df[shuffle(rnd, 1:size(df)[1])[1:expected_requests], :])
    requests = sort(select(requests, [:prod, :ts, :iₚ]), :ts)
    requests.b = [rand(B[i]) for i in requests.iₚ];
    shift_duplicate_ts!(requests)
    h = SimHistory([
        (s = PMDPs.State(pp.c₀, r.ts, r.iₚ), info = (b=r.b,)) for r in eachrow(requests)
        ], 1., nothing, nothing)
end

rnd = MersenneTwister(1)
traces = []
for i in 1:10
    push!(traces, generate_bus_trace(df, pp, rnd))
end
# sampled traces
traces_sam = [traces...]; 

In [171]:
traces = traces_sam

name = "bus_problem"
pp_params = @dict(nᵣ, c, T, expected_requests, res_budget_μ, objective)
data = @dict(name, pp, pp_params, traces)
N = 10
out_folder = "individual"

# Save traces
name = "bus_problem"
sname = savename("traces_bus", pp_params,  "bson")
@tagsave(datadir("traces", sname), @dict(name, pp, pp_params, traces))


dpw_solver = DPWSolver(;depth=50, 
                exploration_constant=40.0, max_time=1.,
                enable_state_pw = false, 
                keep_tree=true, show_progress=false, rng=Xorshift128Plus())

mcts_solver = MCTSSolver(;depth=50, 
                exploration_constant=40.0, max_time=1.,
                rng=Xorshift128Plus())

# PMDPs.process_data(data, PMDPs.flatrate; folder=out_folder, N=N)
# PMDPs.process_data(data, PMDPs.hindsight; folder=out_folder, N=N)
# PMDPs.process_data(data, PMDPs.vi; folder=out_folder, N=N)
# PMDPs.process_data(data, PMDPs.fhvi; folder=out_folder, N=N)
# PMDPs.process_data(data, PMDPs.mcts; folder=out_folder, N=N, method_info="dpw", dpw_solver=mcts_solver)
# PMDPs.process_data(data, PMDPs.mcts; folder=out_folder, N=N, method_info="vanilla", mcts_solver=mcts_solver

┌ Warning: The Git repository is dirty! Appending _dirty to the commit ID
└ @ DrWatson /home/mrkos/.julia/packages/DrWatson/NuoB0/src/saving_tools.jl:60


MCTSSolver(100, 1.0, 50, 40.0, Xorshift128Plus(0x6f1a3202c9a1d501, 0xf62659a7110cb883), RolloutEstimator(POMDPPolicies.RandomSolver(Xorshift128Plus(0x6f1a3202c9a1d501, 0xf62659a7110cb883))), 0.0, 0, false, false)

In [138]:
# generated traces
traces_gen = [PMDPs.simulate_trace(mg, rnd) for i in 1:N]

10-element Array{SimHistory{NamedTuple{(:s, :a, :sp, :r, :info, :t, :action_info),Tuple{PMDPs.State{3},Float64,PMDPs.State{3},Float64,NamedTuple{(:b,),Tuple{Float64}},Int64,Nothing}},Float64},1}:
 [(s = c:[55, 55, 55]_t:1_p:7, a = 1000.0, sp = c:[55, 55, 55]_t:20_p:2, r = 0.0, info = (b = -1.0,), t = 1, action_info = nothing), (s = c:[55, 55, 55]_t:20_p:2, a = 1000.0, sp = c:[55, 55, 55]_t:27_p:2, r = 0.0, info = (b = 13.486347651720473,), t = 2, action_info = nothing), (s = c:[55, 55, 55]_t:27_p:2, a = 1000.0, sp = c:[55, 55, 55]_t:36_p:2, r = 0.0, info = (b = 9.5506308921691,), t = 3, action_info = nothing), (s = c:[55, 55, 55]_t:36_p:2, a = 1000.0, sp = c:[55, 55, 55]_t:135_p:1, r = 0.0, info = (b = 13.031437886525824,), t = 4, action_info = nothing), (s = c:[55, 55, 55]_t:135_p:1, a = 1000.0, sp = c:[55, 55, 55]_t:137_p:1, r = 0.0, info = (b = 16.04596876695922,), t = 5, action_info = nothing), (s = c:[55, 55, 55]_t:137_p:1, a = 1000.0, sp = c:[55, 55, 55]_t:158_p:1, r = 0.0, info 

In [119]:
?zeros

search: zeros count_zeros set_zero_subnormals get_zero_subnormals leading_zeros



```
zeros([T=Float64,] dims::Tuple)
zeros([T=Float64,] dims...)
```

Create an `Array`, with element type `T`, of all zeros with size specified by `dims`. See also [`fill`](@ref), [`ones`](@ref).

# Examples

```jldoctest
julia> zeros(1)
1-element Array{Float64,1}:
 0.0

julia> zeros(Int8, 2, 3)
2×3 Array{Int8,2}:
 0  0  0
 0  0  0
```


In [120]:
default = Dict(zip(1:7, zeros(Int64, 7)))

Dict{Int64,Int64} with 7 entries:
  7 => 0
  4 => 0
  2 => 0
  3 => 0
  5 => 0
  6 => 0
  1 => 0

In [128]:
"""
Get product counts for a trace
"""
function request_counts(tr; n_products=7)
    default = Dict(zip(1:n_products, zeros(Int64, n_products)))
    prod_counts = countmap([s[:s].iₚ for s in tr])
    prod_counts = merge(default, prod_counts) 
    kv = sort(collect(zip(keys(prod_counts), values(prod_counts))))
    [v[2] for v in kv]
end

product_request_counts = hcat(sum([request_counts(tr) for tr in traces_gen])./length(traces_gen),
sum([request_counts(tr) for tr in traces_sam])./length(traces_sam))

7×2 Array{Float64,2}:
 26.2  22.9
 32.2  31.4
  8.2   7.6
  5.9   8.0
  7.3   7.5
  2.0   2.6
  1.0   0.0

In [133]:
sum(product_request_counts, dims=1)

1×2 Array{Float64,2}:
 82.8  80.0

In [164]:
function f(;kwargs...)
    println(kwargs)
    println(haskey(kwargs, "c"))
end

kwargs=Dict(:c=>1, :d=>2)

f(;kwargs...)

Base.Iterators.Pairs(:d => 2,:c => 1)
false
